<h3><center>Reconhecedor de Dígitos numéricos</center></h3>
<h4>Rede Neural Simples</h4>
<li>Projeto de estudo de rede neurais multicamadas (duas).</li>
<li>O desafio apresentado está no link do kaggle, que é uma base de dados de arquivos abertos para estudo de Data Science e pesquisas estatisticas</li>
<li>A proposta do desafio consiste em implementar uma rede neural que seja capaz de identificar digitos escritos a mão em imagens de resulução de 28X28 pixel que são baixados num formato de array com o labe, valores em tons de cinza que variam de 0 (branco) a 1 (preto) compondo a estrutura da matriz que forma a imagem.
<br/><br/>
<center><img src="utils/images/grey_scale_matrix.png"></center>

